# **Анализ важности признаков в логистической регрессии.**

Добро пожаловать в первое домашнее задание курса — посвященное логистической регресии и анализу весов линейной модели в целом. Работать будем с набором данных про диабет! [Источник](https://www.kaggle.com/datasets/mathchi/diabetes-data-set)

### **Предположения:**

Будем полагать, что разведочный анализ данных был проведен заранее и на его основе не выявлено значимых различий признаков по классам.

Однако, важно понимать, что в контексте анализа важности признаков базовое рассмотрение распределений признаков по классам очень значимо. Его может быть достаточно для моделирования новых гипотез о данных и предментной области. При этом также справедливо, что вклады признаков, получаемые на основе статистического анализа и вклады, получаемые на основе моделей машинного обучения могут быть различины.

### **Основное уравнение прогноза:**

$$f(\vec{x}) = \frac{1}{1+exp(-\vec{x})}$$

### **Ход дз:**

В работе мы не будем повторять шаги, изученные на степе, так как с линейными моделями всё просто. Есть коэффициенты — есть интерпретация(при условии стандартизации признаков). Вместо этого, зададимся другим вопросом — достаточно ли точечной оценки весов регрессии? То есть, можно ли, обучив модель на каком-то куске данных и взяв её веса выдвинуть однозначные гипотезы и важности?

**Анализ этих вопросов и ждёт вас в дз. Успешного кодинга!**



Приступим к задаче!

In [ ]:
#Импорт основных библиотек

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
path = 'https://github.com/SadSabrina/interpretable_AI_course/raw/refs/heads/main/data/diabetes.csv'
data = pd.read_csv(path)

data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Сделаем классический шаг для ML — отделим тестовый набор.

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42) #не меняйте random_state

**Примечание:** По хорошему в задаче классификации (и, на самом деле, регрессии) при разбиении на train и test следует помнить об одинаковом (дис)балансе классов. К счастью, sklearn умен и делает это за нас. Этим фактом мы также будем пользоваться при анализе устойчивости весов.

In [ ]:
train_data['Outcome'].value_counts(normalize=True) #убедимся, что sklearn нам не врет

,proportion
Outcome,
0,0.654514
1,0.345486


In [ ]:
test_data['Outcome'].value_counts(normalize=True)

,proportion
Outcome,
0,0.640625
1,0.359375


In [ ]:
#Дальше нормализуем  данные и потом будем работать только с train data
X_train_to_cv, y_train_to_cv = train_data.drop('Outcome', axis=1), train_data['Outcome']

X_test, y_test = test_data.drop('Outcome', axis=1), test_data['Outcome']

Для стандартизации данных воспользуемся StandardScaler, который преобразовывает значение $x_i$ как :

$$x'_i = \frac{(x_i - u)}{s},$$

где $u$ — среднее значение в обучающей выборке, а s— стандартное отклонение.

При интерпретации весов модели вид проводимого преобразования **необходимо понимать**. Иначе связать измненение реального признака с изменением преобразованного не получится.

**Задание 1.**

Что пройдет с преобразованым значением $x'_i$ вектора, если мы увеличим одну из его координат? Статистики $i-го$ признака при этом останутся неизменными.

**Задание 1***

Что пройдет с преобразованым значением $x'_i$ вектора, если мы увеличим одну из его координат? Поведение статистик $i-го$ признака при этом не известно.

In [ ]:
# Дополните преобразование — машстабируйте данные

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train_sc = # Ваш код здесь
X_test_sc = sc.transform(X_test)

Если вы знакомы с машинным обучением достаточно давно, то однозначно встречались с процессом *кросс-валидации.* Мы могли бы обучить модели без этого — но сейчас мы применяем кросс-валидацию для анализа модели.

**Напоминание:**

Процесс кросс-валидации включает в себя следующие шаги:

1. Разделение данных: Исходный набор данных разделяется на K частей, называемых "фолдами". Обычно используется метод k-кросс-валидации, где k - это заданное заранее количество фолдов.
2. Обучение и оценка модели: Затем модель обучается K раз, каждый раз на K-1 фолде данных, а затем оценивается на оставшемся. Таким образом, мы получаем K оценок производительности модели, одну для каждого фолда.
3. Вычисление среднего значения оценок: После завершения всех итераций кросс-валидации вычисляется среднее значение оценок производительности модели. Это дает более стабильную и обобщающую оценку производительности модели. В контексте же весов мы получаем большее понимание о значимости признака на подвыборках.

Здесь мы применяем **стратифицированную кросс-валидацию**. В отличие от классической, для стратифицированной справедливо одинаковое соотношение классов в каждом фолде.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Создание объектов  кросс-валидации. Не меняйте random state
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=12)
kf2 = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
kf3 = StratifiedKFold(n_splits=3, shuffle=True, random_state=13)


Напишем функцию, которая будет:

1. осуществлять кросс-валидацию
2. сохранять коэффициенты регрессий на каждом фолде
3. Позволять обучать логистическую регрессию с разными силами регуляризации (гиперпараметр `C`)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def train_model_and_get_coefs(model, X, y, cross_val):

  coefs = {}
  f1_scores = []


  for iter, (train_idx, val_idx) in enumerate(cross_val.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(X_train, y_train)

    coefs[iter] = model.coef_[0]
    f1_scores.append(f1_score(y_val, model.predict(X_val)))

  print('Mean f1:', np.mean(f1_scores))

  return model, coefs, f1_scores

In [ ]:
model1 = LogisticRegression(C=1, max_iter=1000)
model2 = LogisticRegression(C=0.5, max_iter=1000)
model3 = LogisticRegression(C=0.1, max_iter=1000)

In [ ]:
model1, coefs1, f1_scores1 = train_model_and_get_coefs(model1, X_train_sc, y_train_to_cv, kf)
model2, coefs2, f1_scores2 = train_model_and_get_coefs(model2, X_train_sc, y_train_to_cv, kf2)
model3, coefs3, f1_scores3 = train_model_and_get_coefs(model3, X_train_sc, y_train_to_cv, kf3)

Mean f1: 0.6322746108538905
Mean f1: 0.64030131826742
Mean f1: 0.6083921166260527


**Задание 3.** Обучите лучшую модель с различными объектами кросс-валидации. Какой получился результат? Выберите утверждение на степик.

In [ ]:
model2 = LogisticRegression(C=0.5, max_iter=1000)
model2, coefs2_1, f1_scores2 = # Ваш код здесь


In [ ]:
model2 = LogisticRegression(C=0.5, max_iter=1000)

model2, coefs2_2, f1_scores2_2 = # Ваш код здесь

In [ ]:
model2 = LogisticRegression(C=0.5, max_iter=1000)

model2, coefs2_3, f1_scores2_3 = # Ваш код здесь

**Задание 4.** Подумайте над получившимся результатом. Чем он плох?

`Место для вашего ответа.`

### **Анализ весов:**

Посмотрим теперь на веса, получающиеся на разных кусочках данных для одной модели.

In [ ]:
#Сначала соберем все коэффициенты в одну табличку
coefs1_data = pd.DataFrame(coefs2_1.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])
coefs2_data = pd.DataFrame(coefs2_2.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])
coefs3_data = pd.DataFrame(coefs2_3.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])

In [ ]:
coefs_data = pd.concat([coefs1_data, coefs2_data])
coefs_data = pd.concat([coefs_data, coefs3_data]).reset_index(drop=True)

coefs_data.set_index([['f1', 'f1', 'f1', 'f2', 'f2', 'f2', 'f3', 'f3', 'f3']])

,Pregnancies_weights,Glucose_weights,BloodPressure_weights,SkinThickness_weights,Insulin_weights,BMI_weights,DiabetesPedigreeFunction_weights,Age_weights
f1,0.248665,1.242342,-0.122031,0.034684,-0.189501,0.809906,0.318598,0.462613
f1,0.192625,1.083099,-0.235098,-0.017649,-0.157772,0.813706,0.046859,0.420448
f1,0.167698,0.978066,-0.252655,0.115532,-0.122680,0.627489,0.168529,0.364739
f2,0.233667,1.016411,-0.158403,0.119015,-0.152384,0.660052,0.180026,0.409094
f2,0.059677,1.296335,-0.247994,0.061648,-0.286254,0.752623,0.135169,0.497319
f2,0.309482,1.009224,-0.221707,0.018118,-0.065813,0.776751,0.198391,0.321109
f3,0.060801,0.928017,-0.370929,0.016990,-0.139985,0.842289,0.144557,0.690342
f3,0.349773,1.382352,-0.148400,-0.015519,-0.045662,0.696006,0.242842,0.230546
f3,0.193326,1.000148,-0.151025,0.168476,-0.240217,0.701610,0.146172,0.370101


Обратите внимание как гуляют веса на разных фолдах. Посмотрим на характер "гуляния".

In [ ]:
#Взглянем на статистики полученных весов
coefs_data.describe()

,Pregnancies_weights,Glucose_weights,BloodPressure_weights,SkinThickness_weights,Insulin_weights,BMI_weights,DiabetesPedigreeFunction_weights,Age_weights
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.201746,1.103999,-0.212027,0.055699,-0.155585,0.742270,0.175683,0.418479
std,0.098806,0.161368,0.077108,0.065333,0.076543,0.074754,0.075496,0.128546
min,0.059677,0.928017,-0.370929,-0.017649,-0.286254,0.627489,0.046859,0.230546
25%,0.167698,1.000148,-0.247994,0.016990,-0.189501,0.696006,0.144557,0.364739
50%,0.193326,1.016411,-0.221707,0.034684,-0.152384,0.752623,0.168529,0.409094
75%,0.248665,1.242342,-0.151025,0.115532,-0.122680,0.809906,0.198391,0.462613
max,0.349773,1.382352,-0.122031,0.168476,-0.045662,0.842289,0.318598,0.690342


**Задание 5.** Какой признак является самым важным при всех разбиениях?

**Задание 6.** Какой признак является самым устойчивым при всех разбиениях?

**Подсказка:** он обладает наименьшей дисперсией (и корнем из неё).

### **Уточнение диапазона веса.**

**Рассчет доверительных интервалов.**

Как видите, каждый признак "шатается" от разбиения к разбиению. Однако, некоторые значения могут быть просто связаны с шумом и случайностью. Чтобы построить какую-то более надежную оценку, чем отрезок $[min_{f_i}, max_{fi}]$ построим доверительный инетверал.

**Доверительный интервал** некоторого параметра $\theta$, это интервал, который покрывает неизвестный параметр с заданной надёжностью.

Интерпретировать, например, 95% ДИ можно так:
Если мы будем проводить аналогичное исследование на выборках одного и того же размера много раз, независимо друг от друга то с 95%-ной уверенностью мы можем утверждать, что истинное значение параметра (веса) окажется в диапазоне [w_1, w_2].

**Но 95% интервал не отражает то, что с вероятностью 0.95 истинное значение параметра лежит в найденном диапазоне!**

Для построения можно воспользоваться готовой формулой из scipy `scipy.t.interval`, но можно реализовать и руками. Сделаем оба способа, чтобы подружиться с понятиям.

Формула для вычисления доверительного интервала:
$$\hat{x}-t^*\frac{s}{\sqrt{n}}\leq μ \leq \hat{x}+t^*\frac{s}{\sqrt{n}},$$

где $μ$ искомое значение параметра, \
$t^*$ - значение [t-статистики](http://old.exponenta.ru/educat/referat/XIkonkurs/student5/tabt-st.pdf), соответствующее выбранному уровню доверия (берется из таблицы t-распределения Стьюдента), \
$s$ выборочное стандартное отклонение.

В свою очередь стандартное отклонение генеральной совокупности нам неизвестно, но мы можем его приблизить с помощью стандартного отклонения по выборке:

$s \approx \sqrt{\frac{\sum(x_i-\hat{x})^2}{n-1}}$


In [ ]:
def std(x):

    # Ваш код здесь: реализайте функцию std

    return std


In [ ]:
feature_name = coefs_data.iloc[:, 0].name

x = coefs_data.iloc[:, 0]
s = std(coefs_data.iloc[:, 0])

n = len(x)

In [ ]:
#Для 95% ДИ при 9 наблюдениях 2.26

low = np.mean(x) - 2.26*s/np.sqrt(n)
high = np.mean(x) + 2.26*s/np.sqrt(n)
print(f'{feature_name} ДИ ,({low}, {high})')

Pregnancies_weights ДИ ,(0.12731198238699148, 0.2761801961631034)


Теперь сверимся с библиотечным ДИ.

In [ ]:
from scipy import stats as st

st.t.interval(confidence=0.95, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))

(0.12579681569801826, 0.27769536285207663)

**Задание 7.** Постройте ДИ для всех признаков. Доверительный интервал для какого признака получился самым широким?

**Задание 8.** Доверительный интервал для какого признака получился самым узким? Отметьте, есть ли связь с дисперсией признака по фолдам.

In [ ]:
for i in range(0, 8):
    # Ваш код здесь
    interval = st.t.interval(confidence=0.95, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))

    print('Признак:', name)
    print('95% доверительный интервал:', interval)
    print('Длина ДИ:', round(interval[1]- interval[0], 2))
    print('\n')

**Готово! На этом, первое дз завершено и хочется отметить важные замечания.**

Основная цель данного домашнего задания — знакомство с процессом интерпретации признаков. Составляя его, мы хотели показать, что **точечной оценки веса недостаточно.**

Построение доверительного интервала в данном случае — также лишь начало. В контексте важности можно провести еще более увлекательное исследование, снимая статистики для большего числа итераций кросс-валидации, для большего числа моделей (например, сравнить между собой лес и дерево) и так далее.

Если эта тема кажется вам интересной — дайте знать и мы постараемся подготовить на эту тему дополнительные вебинары! :)


Спасибо за работу!